In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
import json

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

print(sns.__version__)
# Save a palette to a variable:
palette = sns.color_palette("bright")

sns.set_palette(palette)
sns.set(style="whitegrid", font_scale=1.5)

0.11.2


# corona

<div class="alert alert-block alert-info">
<h2> Clustering of outliers </h2>
</div>

In [3]:
CORONA_PATH = "/m/cs/work/luongn1/digirhythm/data/processed/corona/similarity_matrix/"

study = "corona"
with open(f"../config/features.txt") as f:
    features = json.load(f)

freq = "4epochs"
FEATURES = features[study][freq]

# Take a sample
behaviour_vector = pd.read_csv(
    f"../data/processed/corona/vector_corona_{freq}.csv", index_col=0
)
# sample = pd.read_csv(CORONA_PATH + f"{freq}/similarity_{uid}.csv", index_col=0)
similarity_baseline_si = pd.read_csv(
    CORONA_PATH + f"si/similarity_baseline_{freq}.csv", index_col=0
)

if freq == "7ds":
    behaviour_vector.reset_index(inplace=True)
    behaviour_vector.dropna(subset=FEATURES, inplace=True)

similarity_baseline_si.index = similarity_baseline_si.index.astype("str")
behaviour_vector.subject_id = behaviour_vector.subject_id.astype("str")
behaviour_vector.sort_values(["subject_id", "date"], inplace=True)

In [4]:
# Outliers - Days deviating the most from baseline
def outliers(similarity, thres=1.5):
    # Calculate mean and standard deviation
    mean = similarity.mean()
    std_dev = similarity.std()

    # Calculate the lower threshold
    lower_threshold = mean - thres * std_dev

    # Find indexes of outliers
    outliers = similarity[similarity <= lower_threshold].index

    return outliers

In [94]:
outliers_df = pd.DataFrame()
for uid in similarity_baseline_si.index:
    s_si = similarity_baseline_si.loc[uid].dropna()
    outlier_si = s_si.iloc[outliers(s_si)]

    outliers_behav = behaviour_vector[behaviour_vector.subject_id == uid].iloc[
        outliers(s_si)
    ]

    outliers_df = pd.concat([outliers_df, outliers_behav])

outliers_df = outliers_df.reset_index(drop=True)
outliers_df.dropna(inplace=True, subset=FEATURES)
outliers_df

,subject_id,date,tst:sum,tst:min,tst:max,tst:mean,tst:std,midsleep:sum,midsleep:min,midsleep:max,...,steps:morning:7ds:sum:norm,steps:afternoon:7ds:sum:norm,steps:evening:7ds:sum:norm,stepsx1000:total,stepsx1000:night:7ds:sum:norm,stepsx1000:morning:7ds:sum:norm,stepsx1000:afternoon:7ds:sum:norm,stepsx1000:evening:7ds:sum:norm,stepsx1000:total:norm,steps:total:norm
0,38758730,2021-07-07,48.74,6.31,8.39,6.962857,0.769712,193.19,26.12,29.12,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,38758730,2021-07-11,47.74,5.94,7.62,6.820000,0.599194,198.85,26.54,31.54,...,0.260585,0.344366,0.308473,75.530,0.086575,0.260585,0.344366,0.308473,0.539165,0.539165
2,38758730,2021-07-15,47.61,4.49,10.21,6.801429,1.803648,203.99,25.42,31.54,...,0.219121,0.278213,0.356115,79.317,0.146551,0.219121,0.278213,0.356115,0.566198,0.566198
3,38758730,2021-08-14,51.59,5.59,8.92,7.370000,1.071665,193.58,26.36,29.59,...,0.377588,0.343124,0.243879,48.688,0.035409,0.377588,0.343124,0.243879,0.347555,0.347555
4,91109228,2021-08-01,51.18,5.97,8.37,7.311429,0.775402,218.09,30.26,32.71,...,0.117502,0.420100,0.435140,73.335,0.027258,0.117502,0.420100,0.435140,0.349206,0.349206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,4240001906,2021-07-01,46.20,4.76,9.74,6.600000,1.594898,200.72,28.06,29.81,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
279,4240001906,2021-07-03,39.68,4.76,6.94,5.668571,0.823861,199.69,27.86,29.81,...,0.224693,0.330024,0.410993,96.790,0.034291,0.224693,0.330024,0.410993,0.895631,0.895631
280,4240001906,2021-07-04,39.76,4.76,7.02,5.680000,0.844729,198.26,27.86,29.02,...,0.261877,0.361966,0.336133,83.925,0.040024,0.261877,0.361966,0.336133,0.776587,0.776587
281,4270145196,2021-08-21,54.58,5.31,9.14,7.797143,1.251422,189.69,26.68,27.70,...,0.303246,0.514051,0.179123,90.491,0.003580,0.303246,0.514051,0.179123,0.513281,0.513281


In [5]:
# Convert to vector of difference from the means

# Flag the rows
mean_diff = pd.DataFrame()

for uid in similarity_baseline_si.index:
    s_si = similarity_baseline_si.loc[uid].dropna()
    outliers_indices = outliers(s_si)

    outliers_behav = behaviour_vector[behaviour_vector.subject_id == uid].iloc[
        outliers_indices
    ]
    sample_mean = behaviour_vector[behaviour_vector.subject_id == uid].mean()
    
    diff_df = outliers_behav[FEATURES] - sample_mean[FEATURES]
    mean_diff = pd.concat([mean_diff, diff_df])

mean_diff.dropna(inplace=True)
mean_diff

,heart_rate_variability_avg:norm,stepsx1000:total:norm,steps:night:norm,steps:morning:norm,steps:afternoon:norm,steps:evening:norm,tst:norm,midsleep:norm
161,0.258493,0.029292,-0.000756,-0.074724,-0.357731,0.433211,0.044195,0.168992
2344,0.625840,-0.172685,-0.000756,0.275608,-0.195059,-0.079792,0.193534,0.133118
3500,0.074820,0.044122,-0.000634,-0.236811,-0.310417,0.547862,0.012059,-0.153878
3794,0.574820,0.005503,-0.000756,-0.189466,0.350704,-0.160483,-0.038980,-0.131456
4100,0.217677,0.015718,-0.000756,-0.189691,-0.329748,0.520196,-0.048432,-0.109035
...,...,...,...,...,...,...,...,...
26119,0.291530,-0.170728,-0.043443,0.320444,-0.086805,-0.190196,-0.185052,-0.104178
27266,0.496075,-0.064958,-0.044904,-0.035202,-0.116930,0.197037,0.437488,0.055331
27884,0.064257,0.203563,-0.041556,-0.351382,-0.043260,0.436198,0.331943,0.043061
10859,-0.274419,0.159991,-0.000726,-0.221602,0.266230,-0.043901,0.582581,0.418770


In [6]:
from sklearn.metrics import silhouette_score

# Determine optimal cluster number with elbow method
wcss = []

silhouette_scores = []
for i in range(2, 11):
    model = KMeans(
        n_clusters=i,
        init="k-means++",  # Initialization method for kmeans
        max_iter=300,  # Maximum number of iterations
        n_init=10,  # Choose how often algorithm will run with different centroid
        random_state=0,
    )  # Choose random state for reproducibility
    labels = model.fit_predict(mean_diff)

    # Silhouette Index
    silhouette_avg = silhouette_score(mean_diff, labels)
    silhouette_scores.append(silhouette_avg)

# Determine optimal cluster number using maximum Silhouette score and maximum Dunn index
optimal_clusters_silhouette = np.argmax(silhouette_scores) + 2
print("Optimal num cluster: ", optimal_clusters_silhouette)

Optimal num cluster:  4


In [7]:
kmeans = KMeans(
    n_clusters=optimal_clusters_silhouette,  # Set amount of clusters
    init="k-means++",  # Initialization method for kmeans
    max_iter=300,  # Maximum number of iterations
    n_init=10,  # Choose how often algorithm will run with different centroid
    random_state=0,
)  # Choose random state for reproducibility

pred_y = kmeans.fit_predict(mean_diff[FEATURES])

mean_diff["cluster"] = kmeans.labels_
mean_diff.groupby("cluster").mean()

# Denote

# Daily clusters
# 0: Night-active individuals with less day activity and later sleep midpoint.
# 1: Morning-active individuals with earlier sleep times.
# 2: Afternoon-active individuals with higher overall activity and variability.
# 3:Evening-active individuals with increased HRV

# Weekly clusters - Pattern
# 0:
# 1: Significantly reduce steps count, increased hrv

,heart_rate_variability_avg:norm,stepsx1000:total:norm,steps:night:norm,steps:morning:norm,steps:afternoon:norm,steps:evening:norm,tst:norm,midsleep:norm
cluster,,,,,,,,
0,-0.004821,-0.161403,0.016287,0.512385,-0.310942,-0.217730,-0.042132,-0.053984
1,0.075865,0.005673,-0.005978,-0.114192,-0.273662,0.393831,-0.001195,-0.005701
2,-0.097276,-0.213247,0.689408,-0.106399,-0.357465,-0.225544,-0.078750,0.155325
3,0.036984,0.114243,0.010378,-0.061608,0.152183,-0.100953,-0.023014,0.083461
